In [ ]:
# --- SQL Server Connection Test ---
import pyodbc
import pandas as pd

# STEP 1: Check available ODBC drivers
print("🧩 Available ODBC Drivers:")
for driver in pyodbc.drivers():
  print("   ", driver)

# STEP 2: Define connection details
server = "localhost\\SQL2022"              # Change if your instance name differs
database = "QuickStart_DataScience"        # Change to your database
driver = "SQL Server"                      # Use one listed above
#driver = "ODBC Driver 18 for SQL Server"  # Use one listed above
trusted_connection = "yes"                 # or "no" if using SQL auth
encrypt = "no"                             # disable encryption for local

conn_str = ( f"DRIVER={{{driver}}};"
             f"SERVER={server};"
             f"DATABASE={database};"
             f"Trusted_Connection={trusted_connection};"
             f"Encrypt={encrypt};" )

print("\n🔗 Connecting with:")
print(conn_str)

# STEP 3: Test connection and query
try:
  with pyodbc.connect(conn_str) as conn:
    query = "SELECT TOP 20 name, database_id FROM sys.databases;"
    df = pd.read_sql(query, conn)
    print("\n✅ Connection successful! Sample query output:")
    display(df)
except Exception as e:
  print("\n❌ Connection failed!")
  print(str(e))


In [ ]:
from sqlalchemy import create_engine
import pandas as pd

engine = create_engine("mssql+pyodbc://localhost\\SQL2022/QuickStart_DataScience?driver=ODBC+Driver+18+for+SQL+Server&trusted_connection=yes&Encrypt=no")


In [ ]:
# NOTE: 'engine' defined above ...

select_all_from_allstarfull   = "SELECT * FROM dbo.AllstarFull"
select_top10_from_allstarfull = "SELECT TOP 10 * FROM dbo.AllstarFull"

df = pd.read_sql(select_top10_from_allstarfull, engine)
df.head()


In [ ]:
# NOTE: 'engine' defined above ...

select_all_from_people   = "SELECT * FROM dbo.People"
select_top10_from_people = "SELECT TOP 10 * FROM dbo.People"

df = pd.read_sql(select_top10_from_people, engine)
df.head()

In [25]:
# QUESTION: 
# • Connects to an SQL database file and queries for all players who have played at least 50 games and are still
#   active. Use the “finalGame” field from the “People” table to determine if a player is active. Retrieve weight,
#   throws, bats, throws, all birth-related and all name-related columns from the “People” table and retrieve all 
#   columns from the “Batting” table.
# • Converts this data into either an R data frame or a pandas data frame.
# • Adds a calculated column with the player’s age and a calculated column with each player’s first and last name concatenated.
# • Once the calculated columns are added, drops the other columns related to birth date and name.
# • Deletes any rows with missing values

command = """
;WITH active_players AS (
  SELECT
    P.*
  FROM
    dbo.People AS P
  WHERE
    P.deleted = 0 AND 
    ISNULL(P.finalGame, GETDATE()) > '2017/12/31'
),
eligible_players AS (
  SELECT
    A.playerID
  FROM
    dbo.Appearances AS A
  WHERE
    A.deleted = 0
  GROUP BY
    A.playerID
  HAVING
    SUM(CAST(A.G_all AS INT)) >= 50
),
batting_stats AS (
  SELECT
    B.*
  FROM
    dbo.Batting AS B
  WHERE
    B.deleted = 0
)
SELECT
   TRIM(AP.nameFirst) + ' ' + TRIM(AP.nameLast) AS playerName
  ,AP.playerID
--  ,AP.nameFirst
--  ,AP.nameLast
--  ,AP.nameGiven
  ,AP.[weight]
  ,AP.throws
--  ,AP.birthYear
--  ,AP.birthMonth
--  ,AP.birthDay
  ,DATEDIFF(YEAR, DATEFROMPARTS(AP.birthYear, AP.birthMonth, AP.birthDay), GETDATE())
    - CASE 
        WHEN MONTH(GETDATE()) < AP.birthMonth OR
             MONTH(GETDATE()) = AP.birthMonth AND 
               DAY(GETDATE()) < AP.birthDay 
        THEN 1 ELSE 0 
      END AS 'playerAge'
  ,AP.birthCountry
  ,AP.birthState
  ,AP.birthCity
  ,BS.yearID
  ,BS.stint
  ,BS.teamID
  ,BS.lgID
  ,BS.G
  ,BS.AB
  ,BS.R
  ,BS.H
  ,BS._2B
  ,BS._3B
  ,BS.HR
  ,BS.RBI
  ,BS.SB
  ,BS.CS
  ,BS.BB
  ,BS.SO
  ,BS.IBB
  ,BS.HBP
  ,BS.SH
  ,BS.SF
  ,BS.GIDP
FROM active_players AS AP
  INNER JOIN eligible_players AS EP
    INNER JOIN batting_stats AS BS
      ON EP.playerID = BS.playerID
    ON AP.playerID = EP.playerID
ORDER BY 
   AP.playerID  ASC
  ,BS.yearID     DESC
"""

df = pd.read_sql(command, engine)
#df

#NOTE: I am building the 'playerName' within the above SQL.  
# Another way to do it is below:
#df['player_name'] = df['nameFirst'].str.strip() + ' ' + df['nameLast'].str.strip()

#NOTE: I'm also building the 'playerAge' within the above SQL.  
# Another way to do it is below:
from datetime import datetime
def calculate_age(row):
  birth_year  = row['birthYear']
  birth_month = row['birthMonth']
  birth_day   = row['birthDay']

  if pd.isnull(birth_year) or pd.isnull(birth_month) or pd.isnull(birth_day):
    return pd.NA

  today = datetime.today()
  age   = today.year - int(birth_year)

  if (today.month, today.day) < (int(birth_month), int(birth_day)):
    age -= 1
    
  return age

#df['player_age'] = df.apply(calculate_age, axis = 1)
#df

#columns_to_drop = [ 'nameFirst', 'nameLast', 'nameGiven',
#                    'birthYear', 'birthMonth', 'birthDay', 'birthCountry', 'birthState', 'birthCity']
#df = df.drop(columns = columns_to_drop)

# Delete any rows with missing values
#df = df.dropna()

df

,playerName,playerID,weight,throws,playerAge,birthCountry,birthState,birthCity,yearID,stint,...,RBI,SB,CS,BB,SO,IBB,HBP,SH,SF,GIDP
0,Fernando Abad,abadfe01,235,L,39,D.R.,La Romana,La Romana,2023,1,...,0,0,0,0,0,0,0,0,0,0
1,Fernando Abad,abadfe01,235,L,39,D.R.,La Romana,La Romana,2021,1,...,0,0,0,0,0,0,0,0,0,0
2,Fernando Abad,abadfe01,235,L,39,D.R.,La Romana,La Romana,2019,1,...,0,0,0,0,0,0,0,0,0,0
3,Fernando Abad,abadfe01,235,L,39,D.R.,La Romana,La Romana,2017,1,...,0,0,0,0,0,0,0,0,0,0
4,Fernando Abad,abadfe01,235,L,39,D.R.,La Romana,La Romana,2016,1,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13146,Mike Zunino,zuninmi01,235,R,34,USA,FL,Cape Coral,2017,1,...,64,1,0,39,160,0,8,0,1,8
13147,Mike Zunino,zuninmi01,235,R,34,USA,FL,Cape Coral,2016,1,...,31,0,0,21,65,0,6,0,1,0
13148,Mike Zunino,zuninmi01,235,R,34,USA,FL,Cape Coral,2015,1,...,28,0,1,21,132,0,5,8,2,6
13149,Mike Zunino,zuninmi01,235,R,34,USA,FL,Cape Coral,2014,1,...,60,0,3,17,158,1,17,0,4,12
